In [1]:
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01m


In [2]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)


In [3]:
!pip install pandas

  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 KB 116.1 kB/s eta 0:00:001m110.7 kB/s eta 0:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


In [24]:
from collections import defaultdict

from surprise import Dataset, SVD


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [23]:
df=pd.read_csv('xaa')
df=df.drop(columns=['is_read','is_reviewed'])
df

,user_id,book_id,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5
...,...,...,...
99994,227,17543,2
99995,227,882,3
99996,227,59168,3
99997,227,24428,3


In [1]:
import pandas as pd

from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {
    "itemID": [1, 1, 1, 2, 2],
    "userID": [9, 32, 2, 45, "user_foo"],
    "rating": [3, 2, 4, 3, 1],
}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.41421356, 1.8225456 ]),
 'test_mae': array([1.33333333, 1.81291835]),
 'fit_time': (0.0016832351684570312, 3.0994415283203125e-05),
 'test_time': (8.630752563476562e-05, 1.621246337890625e-05)}

In [2]:
from surprise import KNNBasic
algo = KNNBasic()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0
...,...,...,...,...,...
99994,227,17543,1,2,0
99995,227,882,1,3,0
99996,227,59168,1,3,0
99997,227,24428,1,3,0


,user_id,book_id,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5
...,...,...,...
99994,227,17543,2
99995,227,882,3
99996,227,59168,3
99997,227,24428,3


In [15]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)

In [16]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [17]:
from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [18]:
predictions = algo.test(testset)

In [19]:
from surprise import accuracy
accuracy.rmse(predictions)


RMSE: 1.7661


1.7660930291743344

In [21]:
algo.predict

<bound method AlgoBase.predict of <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f9c426e2860>>

In [26]:
get_top_n(predictions,n=10)[220]

[(862, 4.195318865100763),
 (4513, 3.934986149269138),
 (46272, 3.8277798843690336),
 (997, 3.756987077747328),
 (998, 3.6936619815160365),
 (821, 3.513512803545891),
 (846, 3.490909825711597),
 (968, 3.4729663515997427),
 (17591, 3.4123699669542336),
 (46259, 3.3073872707741385)]